In [ ]:
import os
import pandas as pd
import random
import json
from dotenv import load_dotenv
from jinja2 import Template
import openai
import openpyxl

In [ ]:
%load_ext dotenv
%dotenv
dotenv_path = ('.env')

In [ ]:
load_dotenv(dotenv_path = dotenv_path, override = True)
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_VERSION")
openai.api_base = os.getenv("AZURE_ENDPOINT")

In [ ]:
# Import Logic Scenarios
logic_scenarios = '../data/logic_input/logic_scenarios.xlsx'

# Import Logic Prompt
logic_prompt = '../data/logic_prompts/logic_prompts.jinja2'
filled_prompt = '../data/logic_prompts/filled_logic_prompt.jinja2'

## Logic Prompt Experiments

This notebook explores three types of experiments with the purpose of mitigating bias while helping LLMs self reasonn using prompts.

### Symbolic Logic and Counterfactual Data

The first set of experiments explore if LLMs can be taught symbolic logic to better reason AND explores if identity of a sentence's subject causes the LLM to reason differently. This will be compared to how the LLM reasons without the symbolic logic prompt. 

In [ ]:
# Counterfactual Examples

GENDER = ['Gender-fluid', 'Nonbinary', 'Man', 'Woman', 'Transgender', 'Two Spirit', 'Intersex'] # By no means exhaustive list of gender identities and expressions
RACE = ['Black', 'White', 'Hispanic', 'Middle Eastern', 'Asian', 'Native American', 'Pacific Islander'] # While Hispanic is an ethnicity, added to list for the sake of this exercise
BLANK = ['Good', 'Bad']
logic_df = pd.read_excel(logic_scenarios)
logic_list = logic_df['scenarios'].values.tolist()

In [ ]:
random_gender = random.choice(GENDER)
random_race = random.choice(RACE)
random_blank = random.choice(BLANK)

In [ ]:
# Collect Prompt and Template
with open(f'{logic_prompt}', 'r') as logic_file:
    logic_template = Template(logic_file.read())

render_logic = logic_template.render(
    Gender = random_gender,
    Race = random_race,
    Blank = random_blank 
)
# Output responses in a list
res_list = []

for i in range(len(logic_list)):
    response = openai.ChatCompletion.create(
        engine=engine, # Change to deployment name
        messages = [{"role":"system","content": render_logic},
                    {'role': 'user', 'content': logic_list[i]}
    ], 
        temperature=0,
        max_tokens=350,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
    res = response['choices'][0]['message']['content']
    res_list.append(res)
    
print(res_list)

In [ ]:
with open('../data/logic_input/nonfilled_list.json', 'w') as notfilled_file:
    json.dump(res_list, notfilled_file)

#### Running With Tags Pre-Filled

In [ ]:
filled_list = []
for logic in logic_list:
    x = str(logic)
    filled = x.format(RACE = random.choice(RACE), GENDER= random.choice(GENDER), BLANK = random.choice(BLANK))
    filled_list.append(filled)
filled_list

In [ ]:
# Collect Prompt and Template
with open(f'{filled_prompt}', 'r') as filled_file:
    filled_template = Template(filled_file.read())

render_filled = filled_template.render()

# Collect all responses
filled_log_list = []

for i in range(len(filled_list)):
    response = openai.ChatCompletion.create(
        engine=engine, # Change to deployment name
        messages = [{"role":"system","content": render_filled},
                    {'role': 'user', 'content': filled_list[i]}
    ], 
        temperature=0,
        max_tokens=350,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
    res = response['choices'][0]['message']['content']
    filled_log_list.append(res)
    
print(filled_log_list)

In [ ]:
with open('../data/logic_input/filled_list.json', 'w') as filled_file:
    json.dump(filled_log_list, filled_file)

In [ ]:
noprompt_list = []
for i in range(len(filled_list)):
    response = openai.ChatCompletion.create(
        engine=engine, # Change to deployment name
        messages = [
                    {'role': 'user', 'content': filled_list[i]}
    ], 
        temperature=0,
        max_tokens=350,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
    res = response['choices'][0]['message']['content']
    noprompt_list.append(res)
print(noprompt_list)

In [ ]:
with open('../data/logic_input/no_prompt.json', 'w') as noprompt_file:
    json.dump(noprompt_list, noprompt_file)